In [9]:
import pandas as pd
import altair as alt
import numpy as np


In [33]:
data = pd.read_csv("adult.data", usecols = [i for i in range(14)])

In [34]:
data.columns = ["age", "workclass", "wage", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "native_country"]

In [100]:
data = data.head(5000)

In [115]:
data.head(1000)

,age,workclass,wage,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,45,Private,89325,Masters,14,Divorced,Prof-specialty,Not-in-family,White,Male,0,0,45,United-States
996,48,Federal-gov,33109,Bachelors,13,Divorced,Exec-managerial,Unmarried,White,Male,0,0,58,United-States
997,40,Private,82465,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,2580,0,40,United-States
998,39,Self-emp-inc,329980,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,15024,0,50,United-States


In [102]:
points = alt.Chart(data).mark_circle().encode(
   alt.X(alt.repeat("column"), type = "quantitative"),
   alt.Y(alt.repeat("row"), type = "quantitative"),
   color='sex:N',
).properties(
    width = 150,
    height = 150,
).repeat(
    row = ["age", "wage", "hours_per_week"],
    column = ["hours_per_week", "wage", "age"]).interactive()

In [103]:
points

alt.RepeatChart(...)

In [104]:
bars = alt.Chart(data).mark_bar().encode(
    y='sex:N',
    x='mean(wage):Q',
    color = "sex:N"
).interactive()

In [105]:
bars

alt.Chart(...)

In [106]:
alt.Chart(data).mark_rect().encode(
    x="education:N",
    y="workclass:N",
    color = "wage:Q"
)

alt.Chart(...)

In [113]:
scatter = alt.Chart(data).mark_circle().encode(
   x = "age:Q",
   y = "wage:Q",
   color = "sex:N",
   tooltip = ["age", "marital_status", "wage"]
).properties(
    width = 350,
    height = 350,
).interactive()

In [114]:
scatter

alt.Chart(...)

In [109]:
alt.Chart(data).mark_bar().encode(
    x=alt.X("education:N",),
    y=alt.Y("workclass:N"),
    color = "wage:Q"
)

alt.Chart(...)

In [127]:
alt.Chart(data).mark_bar().encode(
    alt.X("occupation:N"),
    y='mean(wage):Q',
).configure_mark(
    opacity = 0.3,
    color = "green"
)

alt.Chart(...)